In [ ]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

#### Para testear las funciones, levanto los dataframes originales

In [ ]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
year = 2018

In [ ]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    return temp

In [ ]:
def _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_campo_user(df, campo)

In [ ]:
def _sum_total_user(df_pageviews, campo):
    return df_pageviews.groupby("USER_ID")[campo].sum()

In [ ]:
def _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_total_user(df, campo)

In [ ]:
def _sum_campo_user_1m(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, 1)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_campo_user_15d(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 15)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_campo_user_7d(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 7)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_campo_user_1d(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 1)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_total_user_1m(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, 1)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_total_user_15d(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 15)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_total_user_7d(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 7)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _sum_total_user_1d(df_pageviews, mes_snapshot, campo):
    fecha_desde = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 1)
    fecha_hasta = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    return _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo)

In [ ]:
def _ratio_page_all(df_pageviews, mes_snapshot, campo):
    df_1m = _sum_campo_user_1m(df_pageviews, mes_snapshot, campo)
    df_15d = _sum_campo_user_15d(df_pageviews, mes_snapshot, campo)
    df_7d = _sum_campo_user_7d(df_pageviews, mes_snapshot, campo)
    df_1d =_sum_campo_user_1d(df_pageviews, mes_snapshot, campo)
    df_ever =_sum_campo_user(df_pageviews, campo)

    tot_1m = _sum_total_user_1m(df_pageviews, mes_snapshot, campo)
    tot_15d = _sum_total_user_15m(df_pageviews, mes_snapshot, campo)
    tot_7d = _sum_total_user_17m(df_pageviews, mes_snapshot, campo)    
    tot_1d = _sum_total_user_1d(df_pageviews, mes_snapshot, campo)
    tot_ever = _sum_total_user(df_pageviews, campo)

    return df_1m,df_15d, df_7d, df_1d, df_ever, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever

In [ ]:
def _ratio_all(df_pageviews, mes_snapshot, campo):
    df_1m,df_15d, df_7d, df_1d, df_ever, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever =_ratio_page_all(df_pageviews, mes_snapshot, campo)
    
    ratio_preferencia_1m = df_1m.div(tot_1m, axis=0)
    ratio_preferencia_15d = df_15d.div(tot_15d, axis=0)
    
    
    #completar 
    
    
    
    return ratio_preferencia_1m, ratio_preferencia_15d

In [ ]:
def ratio_page_all(df_pageviews, mes_snapshot):
    campo = 'PAGE'
    return ratio_all(df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_ccategory_all(df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return ratio_all(df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_cctop_all(df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return ratio_all(df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_ccbottom_all(df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return ratio_all(df_pageviews, mes_snapshot, campo)